In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots

### Data

In [2]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [3]:
cell_demand = 0.001*164.98*(1.369*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [4]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [5]:
input_amount = []
input_amount = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount

MktV = price .* input_amount ;  # ton

In [6]:
input_amount

15-element Vector{Float64}:
  49670.291589732
  69512.470736544
  67696.846500888
  69382.78329114
 129687.44540400001
  72376.074329
   5239.896784
   1309.974196
  31111.887154999997
  60258.81301599999
 327493.549
   2258.5762
   1806.86096
 451715.24
 451715.24

----

In [140]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [141]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)   # y[i,j,k] ship product k from i to j 
@variable(model, delta[1:ncty], Bin);

In [142]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end


# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end


# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end


# LCA: As=f
for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)


# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end

In [143]:
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = (price[k]/vGDP[i]) * sink_c[i] + (price[k]/global_gdp) * global_sink
    end
end
proS = (x.*pro_sink)*ones(nproc,1) 


############################
tranD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
end


###################################
Allo_soc = proM ./ vGDP .* Dsoc


#############
vec_S = proS
vec_D = proD + tranD
vec_Dsoc = Allo_soc;

In [144]:
threshold = 0.1
UB = vec_S - threshold * (vec_S-vec_Dsoc)
LB = vec_Dsoc + threshold * (vec_S-vec_Dsoc);

In [145]:
# M = 1e12
# for i in 1:ncty
#     @constraint(model, vec_D[i]-LB[i] >= (delta[i] - 1) * M)
#     @constraint(model, vec_D[i]-LB[i] <= delta[i] * M)
#     @constraint(model, UB[i]-vec_D[i] >= (delta[i] - 1) * M)
#     @constraint(model, UB[i]-vec_D[i] <= delta[i] * M)
# end


M = 1e10
slack = 0.0001
for i in 1:ncty
#     @constraint(model, vec_D[i]-LB[i] >= (delta[i] - 1) * M)
#     @constraint(model, vec_D[i]-LB[i] <= delta[i] * M)
    @constraint(model, UB[i]-vec_D[i] >= slack + (delta[i] - 1) * M)
    @constraint(model, UB[i]-vec_D[i] <= delta[i] * M)
end

In [146]:
@objective(model, Max, sum(delta));

In [147]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2265 rows, 13980 columns and 55800 nonzeros
Model fingerprint: 0xd45673aa
Variable types: 13950 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1914 rows and 12387 columns
Presolve time: 0.03s
Presolved: 351 rows, 1593 columns, 6203 nonzeros
Variable types: 1566 continuous, 27 integer (27 binary)

Root relaxation: objective 2.798029e+01, 358 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.98029    0    2          -   27.98

In [94]:
soc_opt_x = JuMP.value.(x)
soc_opt_y = JuMP.value.(y)
soc_opt_delta = JuMP.value.(delta)
soc_opt = JuMP.objective_value(model);

In [95]:
res_x = DataFrame(soc_opt_x, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries)

Row,country,Li,Co,Mn,Ni,cathode,graphite,PP,PE,Cu,Al,cell,PET,electronics,noncell,battery
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,379.315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2258.58,0.0,0.0,0.0
2,Australia,27088.2,0.0,0.0,0.0,0.0,500.0,0.0,0.0,0.0,7292.98,0.0,0.0,0.0,0.0,0.0
3,Brazil,1802.74,0.0,64035.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Canada,500.0,3100.0,0.0,68843.7,0.0,15000.0,0.0,0.0,0.0,30124.4,0.0,0.0,0.0,0.0,0.0
5,Chile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,China,19000.0,0.0,0.0,0.0,69657.6,26806.6,5121.01,0.0,0.0,0.0,3.26727e5,0.0,0.0,0.0,0.0
7,Colombia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Congo,0.0,53458.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
# CSV.write("/Users/bourg/Desktop/new_obj_1009.csv", res_x) 

### Test

In [96]:
proD = (soc_opt_x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)
proM = soc_opt_x * price;

In [97]:
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = (price[k]/vGDP[i]) * sink_c[i] + (price[k]/global_gdp) * global_sink
    end
end
proS = (soc_opt_x.*pro_sink)*ones(nproc,1);

In [98]:
tranD = zeros(ncty,1)
for j in 1:ncty
    arc_emi = 0
    for i in 1:ncty
        amount = sum(soc_opt_y[i,j,k] for k in 1:nproc)
        arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
    end
    tranD[j] = arc_emi
end

In [99]:
Allo_soc = proM ./ vGDP .* Dsoc;

In [100]:
vec_S = proS
vec_D = proD + tranD
vec_Dsoc = Allo_soc;

In [101]:
threshold = 0.1
UB = vec_S - threshold * (vec_S - vec_D)
LB = vec_Dsoc + threshold * (vec_S - vec_D);

In [103]:
vec_D

30×1 Matrix{Float64}:
   7946.047500563336
 300186.9038896368
 184561.16973597236
      1.4412350207338766e6
      0.0
      1.5328439956254172e6
      0.0
      1.359746940200916e6
      0.0
      0.0
      0.0
    549.5516336327978
 455232.15615241084
      ⋮
  11543.042580000001
      0.0
      0.0
      0.0
 958661.9022711741
      0.0
      0.0
      4.5031692642146135e6
      0.0
  32645.14832932916
      0.0
 184567.13713330677

In [104]:
LB

30×1 Matrix{Float64}:
    661.7959479010133
  24351.341529640762
   8707.533671623736
  27480.612520226834
      0.0
 357026.14068958844
      0.0
      1.2731123184929075e7
      0.0
      0.0
      0.0
    746.0229743600659
  61176.03990987667
      ⋮
   4477.700339218834
      0.0
      0.0
      0.0
      3.3721650440336734e7
      0.0
      0.0
      2.301997853230893e8
      0.0
   5433.4890106869025
      0.0
   8249.935199623751

In [107]:
proM[8], vGDP[8]

(4.249948423555248e9, 5.54e10)

In [140]:
threshold = 0.1
UB = Supply - threshold * (Supply - Demand_soc)
LB = Demand_soc + threshold * (Supply - Demand_soc);

In [142]:
count = 0
for i in 1:ncty
    if Demand[i] <= UB[i] &&  Demand[i] >= LB[i]
        count += 1
    else
        println(i)
    end
end
        

2
30


In [146]:
UB[2], Demand[2], LB[2]

(90933.1734161161, 95533.73970084239, 16587.209657751875)

In [147]:
UB[30], Demand[30], LB[30]

(3.3370216879528533e6, 5.724007314383266e6, 512667.9505599628)

In [148]:
UB[1], Demand[1], LB[1]

(23577.0312852324, 21401.3109498063, 4108.874720680133)

In [151]:
1.98e8 / 179000000

1.106145251396648

In [150]:
emi_sink

Row,country,emission,sink ton/yr
,String31,Float64,Int64
1,Argentina,1.98e8,179000000
2,Australia,3.89e8,287000000
3,Brazil,8.19e8,1810000000
4,Canada,6.15e8,1360000000
5,Chile,3.35e7,275000000
6,China,1.01e10,749000000
7,Colombia,1.64e8,242000000
8,Congo,6.82e8,1500000000
9,Finland,4.77e7,76800000
